# 基于 Census Tract Block 的 TAZ 内部距离计算（Notebook 版 - 生成 i=j 内部距离）

本 notebook 将脚本改写为可交互运行的 `ipynb` 版本（去除 `argparse`，用参数单元格控制）：
- 读取 **LODES**（block 级 OD）、**Census Block**（shapefile）、**zones**（包含 `zone` 列）；
- 计算每个 **TAZ** 的内部 block 间距离统计（km）；
- 生成 `intra_zone_distances`（i=j），并保存为 Parquet。

> 依赖：`pandas`, `geopandas`, `numpy`, `shapely`


In [40]:
# === 参数设置（根据需要修改） ===
lodes_path = "../data/raw data/lodes.csv"                      # LODES 数据路径
blocks_path = "../data/raw data/tl_2022_48_tabblock20.shp"     # Block shapefile 路径
zones_path = "../data/zones.csv"                               # 区域列表（需包含 'zone' 列）
output_path = "../data/intra_zone_distances.parquet"           # 输出路径
stat_method = "mean"                                        # 可选: "mean" | "median" | "min" | "max"
analyze_only = False                                        # True 则仅分析，不生成/保存结果


In [41]:
# === 导入依赖 ===
import pandas as pd
import geopandas as gpd
import numpy as np
from pathlib import Path
from typing import Dict, Tuple
from shapely.geometry import Point
import math

pd.set_option("display.max_columns", 100)


## 函数定义

In [42]:
def load_and_prepare_data(
    lodes_path: str,
    blocks_path: str
) -> Tuple[pd.DataFrame, gpd.GeoDataFrame]:
    """加载和准备 LODES 和 Census Block 数据。"""
    print("加载 LODES 数据...")
    lodes = pd.read_csv(lodes_path)
    
    print("加载 Census Tract Block 数据...")
    blocks = gpd.read_file(blocks_path)
    
    # 类型清洗
    blocks['GEOID20'] = blocks['GEOID20'].astype(str)
    lodes['w_geocode'] = lodes['w_geocode'].astype(str)
    lodes['h_geocode'] = lodes['h_geocode'].astype(str)
    lodes['w_zone'] = lodes['w_zone'].astype(str).str.replace('.0', '', regex=False)
    lodes['h_zone'] = lodes['h_zone'].astype(str).str.replace('.0', '', regex=False)
    
    print(f"LODES 数据: {len(lodes)} 条记录")
    print(f"Block 数据: {len(blocks)} 个 block")
    return lodes, blocks


In [43]:
def calculate_block_centroids(blocks_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """计算每个 block 的质心坐标（经纬度）。"""
    print("计算 block 质心...")
    blocks = blocks_gdf.copy()
    if blocks.crs is None:
        raise ValueError("Block 数据缺少 CRS（坐标参考系）。请确保 shapefile 带有正确的 crs。")
    
    if blocks.crs.is_geographic:
        projected = blocks.to_crs('EPSG:32614')  # UTM Zone 14N（可按地区调整）
        projected['centroid'] = projected.geometry.centroid
        blocks['centroid'] = projected['centroid'].to_crs(blocks.crs)
    else:
        blocks['centroid'] = blocks.geometry.centroid
    
    blocks['centroid_lat'] = blocks['centroid'].y
    blocks['centroid_lon'] = blocks['centroid'].x
    return blocks


In [44]:
def haversine_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """计算两点间的 Haversine 距离（公里）。"""
    R = 6371.0088  # 地球半径 km
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    return 2 * R * math.asin(math.sqrt(a))


In [45]:
def calculate_intra_taz_block_distances(
    lodes_df: pd.DataFrame,
    blocks_gdf: gpd.GeoDataFrame
) -> Dict[str, Dict]:
    """计算每个 TAZ 内部基于 block 的距离统计。"""
    print("计算 TAZ 内部 block 距离...")
    intra_taz_od = lodes_df[lodes_df['w_zone'] == lodes_df['h_zone']].copy()
    print(f"TAZ 内部 OD 记录数: {len(intra_taz_od)}")
    
    # block -> (lat, lon)
    block_coords = {
        row['GEOID20']: {'lat': row['centroid_lat'], 'lon': row['centroid_lon']}
        for _, row in blocks_gdf.iterrows()
    }
    
    stats: Dict[str, Dict] = {}
    for zone_id in intra_taz_od['w_zone'].unique():
        zone_od = intra_taz_od[intra_taz_od['w_zone'] == zone_id]
        distances = []
        valid_pairs = 0
        
        for _, r in zone_od.iterrows():
            w_geocode = r['w_geocode']
            h_geocode = r['h_geocode']
            if w_geocode == h_geocode:
                continue
            if w_geocode in block_coords and h_geocode in block_coords:
                w = block_coords[w_geocode]
                h = block_coords[h_geocode]
                d = haversine_distance(w['lat'], w['lon'], h['lat'], h['lon'])
                distances.append(d)
                valid_pairs += 1
        
        if distances:
            stats[str(zone_id)] = {
                'mean_distance': float(np.mean(distances)),
                'median_distance': float(np.median(distances)),
                'min_distance': float(np.min(distances)),
                'max_distance': float(np.max(distances)),
                'std_distance': float(np.std(distances)),
                'count_pairs': int(len(distances)),
                'valid_pairs': int(valid_pairs),
            }
        else:
            stats[str(zone_id)] = {
                'mean_distance': 0.5,
                'median_distance': 0.5,
                'min_distance': 0.5,
                'max_distance': 0.5,
                'std_distance': 0.0,
                'count_pairs': 0,
                'valid_pairs': 0,
            }
    return stats


In [46]:
def create_intra_zone_distances(
    zones_df: pd.DataFrame,
    taz_distance_stats: Dict[str, Dict],
    method: str = "mean"
) -> pd.DataFrame:
    """根据 TAZ 统计生成 i=j 的内部距离 DataFrame。"""
    print(f"创建基于 block 的 TAZ 内部距离（{method}）...")
    intra_distances = []
    
    for _, zone_row in zones_df.iterrows():
        # 与脚本一致：将 zone 转成 int 再转 str，避免 '317.0' 之类
        zone_id = str(int(zone_row['zone']))
        padded_zone = zone_id.zfill(8)  # 仅补齐 i/j 到 8 位
        
        if zone_id in taz_distance_stats:
            stats = taz_distance_stats[zone_id]
            if method == "mean":
                distance = stats['mean_distance']
            elif method == "median":
                distance = stats['median_distance']
            elif method == "min":
                distance = stats['min_distance']
            elif method == "max":
                distance = stats['max_distance']
            else:
                distance = stats['mean_distance']
            
            time_minutes = (distance / 30.0) * 60  # 假设平均速度 30 km/h
            intra_distances.append({
                'i': padded_zone,
                'j': padded_zone,
                'dist_km': distance,
                'base_minutes': time_minutes
            })
            
            if zone_id in ['317', '319', '320']:
                print(f"TAZ {zone_id}: 距离={distance:.3f} km, 时间={time_minutes:.2f} 分钟, 有效对={stats['valid_pairs']}")
        else:
            intra_distances.append({
                'i': padded_zone,
                'j': padded_zone,
                'dist_km': 0.5,
                'base_minutes': 0.86
            })
            if zone_id in ['308', '317', '319', '320']:
                print(f"TAZ {zone_id}: 使用默认值 0.5 km (不在统计中)")
    
    return pd.DataFrame(intra_distances)


## 运行：加载数据与计算质心

In [47]:
lodes_df, blocks_gdf = load_and_prepare_data(lodes_path, blocks_path)
zones_df = pd.read_csv(zones_path)
blocks_with_centroids = calculate_block_centroids(blocks_gdf)
print("完成：数据与质心准备。")

加载 LODES 数据...


加载 Census Tract Block 数据...
LODES 数据: 1327205 条记录
Block 数据: 668757 个 block
计算 block 质心...
完成：数据与质心准备。


## 运行：计算 TAZ 内部基于 block 的距离统计

In [48]:
taz_distance_stats = calculate_intra_taz_block_distances(lodes_df, blocks_with_centroids)

print("\n=== TAZ 内部 Block 距离统计（概览） ===")
valid_tazs = [z for z in taz_distance_stats.values() if z['valid_pairs'] > 0]
print(f"有有效数据的 TAZ 数量: {len(valid_tazs)}")
if valid_tazs:
    all_means = [t['mean_distance'] for t in valid_tazs]
    print(f"平均距离范围: {np.min(all_means):.3f} - {np.max(all_means):.3f} km")
    print(f"总体平均距离: {np.mean(all_means):.3f} km")


计算 TAZ 内部 block 距离...
TAZ 内部 OD 记录数: 20499

=== TAZ 内部 Block 距离统计（概览） ===
有有效数据的 TAZ 数量: 534
平均距离范围: 0.080 - 4.701 km
总体平均距离: 0.817 km


## 运行：生成 i=j 的 TAZ 内部距离并保存（可选）

In [49]:
if not analyze_only:
    intra_distances_df = create_intra_zone_distances(zones_df, taz_distance_stats, stat_method)
    print("\n=== 结果分析 ===")
    print(f"TAZ 内部距离范围: {intra_distances_df['dist_km'].min():.3f} - {intra_distances_df['dist_km'].max():.3f} km")
    print(f"TAZ 内部平均距离: {intra_distances_df['dist_km'].mean():.3f} km")
    print(f"TAZ 内部平均时间: {intra_distances_df['base_minutes'].mean():.3f} 分钟")
    print(f"TAZ 内部距离标准差: {intra_distances_df['dist_km'].std():.3f} km")
    
    # 保存结果（Parquet + CSV）
    print(f"保存到: {output_path}")
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    intra_distances_df.to_parquet(output_path, index=False)

    csv_path = str(Path(output_path).with_suffix(".csv"))
    intra_distances_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    
    print(f"完成：保存结果 (parquet + csv)。")
else:
    print("仅分析：未生成/保存 i=j 内部距离。")


创建基于 block 的 TAZ 内部距离（mean）...
TAZ 308: 使用默认值 0.5 km (不在统计中)
TAZ 317: 距离=0.311 km, 时间=0.62 分钟, 有效对=20
TAZ 319: 距离=0.721 km, 时间=1.44 分钟, 有效对=52
TAZ 320: 距离=0.558 km, 时间=1.12 分钟, 有效对=36

=== 结果分析 ===
TAZ 内部距离范围: 0.080 - 1.175 km
TAZ 内部平均距离: 0.442 km
TAZ 内部平均时间: 0.832 分钟
TAZ 内部距离标准差: 0.165 km
保存到: ../data/intra_zone_distances.parquet
完成：保存结果 (parquet + csv)。
